<a href="https://colab.research.google.com/github/xyhuang7215/2021VRDL-final/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Install mmcv & [mmclassfication](https://github.com/open-mmlab/mmclassification)


In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html &> /dev/null
!pip install git+https://github.com/open-mmlab/mim.git &> /dev/null
!mim install mmcls &> /dev/null

In [ ]:
!git clone https://github.com/open-mmlab/mmclassification.git &> /dev/null
%cd mmclassification
!pip install -e . &> /dev/null

Download the cropped data

In [ ]:
!pip install gdown==4.2.0 &> /dev/null
!mkdir crop
%cd crop
!gdown https://drive.google.com/uc?id=1-21kEFvSJ6KrQSdD0-PdID-CIsVz0uCP
!gdown https://drive.google.com/uc?id=1-2ZyTHaRLCa5RbnclqaGrf0eZ4_RQRxf
!gdown https://drive.google.com/uc?id=1-2Ukm8-bQJZNejDvytqQeop6FoFMOqUS
!unzip train_crop.zip &> /dev/null
!unzip test_st1.zip &> /dev/null
!unzip test_st2.zip &> /dev/null
%cd .. 

Download the train/test imgs from kaggle

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

# Fill in your kaggle info
api_token = {"username":"kent7215","key":"3065ba9573cd023aab1f637f1fae793e"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json
!mkdir dataset
%cd dataset
!kaggle competitions download -c the-nature-conservancy-fisheries-monitoring
!unzip train.zip &> /dev/null
!unzip test_stg1.zip &> /dev/null
!7z x ./test_stg2.7z &> /dev/null
!gdown https://drive.google.com/uc?id=1YtUoLWseXUQfBQ_G7_NSbXj-lFPxBqJZ
%cd ..

Download the checkpoints and configs

In [ ]:
!gdown https://drive.google.com/drive/folders/1ShJcNI2mwj39sGj5DF-yZIEDWzQbeXuB? -O ./ --folder

You may download the pretrained model and train by yourself

In [ ]:
# !mkdir checkpoints
# !wget -c https://download.openmmlab.com/mmclassification/v0/swin-transformer/swin_small_224_b16x64_300e_imagenet_20210615_110219-7f9d988b.pth \
#       -O checkpoints/pretrained_small.pth
# !wget -c https://download.openmmlab.com/mmclassification/v0/resnest/resnest101_imagenet_converted-032caa52.pth \
#       -O checkpoints/pretrained2.pth
# !wget -c https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_batch256_imagenet_20200708-cfb998bf.pth \
#        -O checkpoints/resnet_pretrained.pth
# !python tools/train final/swin.py

In [ ]:
import os
import csv
import warnings

import mmcv
import numpy as np
import torch
from mmcv.parallel import collate, scatter
from mmcv.runner import load_checkpoint

from mmcls.datasets.pipelines import Compose
from mmcls.models import build_classifier
from mmcls.apis import inference_model, init_model, show_result_pyplot


def my_inference_model(model, img):
    """Inference image(s) with the classifier.

    Args:
        model (nn.Module): The loaded classifier.
        img (str/ndarray): The image filename or loaded image.

    Returns:
        result (dict): The classification results that contains
            `class_name`, `pred_label` and `pred_score`.
    """
    cfg = model.cfg
    device = next(model.parameters()).device  # model device
    # build the data pipeline
    if isinstance(img, str):
        if cfg.data.test.pipeline[0]['type'] != 'LoadImageFromFile':
            cfg.data.test.pipeline.insert(0, dict(type='LoadImageFromFile'))
        data = dict(img_info=dict(filename=img), img_prefix=None)
    else:
        if cfg.data.test.pipeline[0]['type'] == 'LoadImageFromFile':
            cfg.data.test.pipeline.pop(0)
        data = dict(img=img)
    test_pipeline = Compose(cfg.data.test.pipeline)
    data = test_pipeline(data)
    data = collate([data], samples_per_gpu=1)
    if next(model.parameters()).is_cuda:
        # scatter to specified GPU
        data = scatter(data, [device])[0]

    # forward the model
    with torch.no_grad():
        scores = model(return_loss=False, **data)
        pred_score = np.max(scores, axis=1)[0]
        pred_label = np.argmax(scores, axis=1)[0]
        result = {'pred_label': pred_label, 'pred_score': float(pred_score)}
    result['pred_class'] = model.CLASSES[result['pred_label']]
    return scores[0]

In [ ]:
def write_csv(model, csv_name, test_prefix):
    with open(csv_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['image', 'ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])

    folder = os.path.join(test_prefix, 'test_stg1')
    imgs = os.listdir(folder)
    imgs.sort()
    with open(csv_name, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for img in imgs:
            result = list(my_inference_model(model, os.path.join(folder, img)))
            result.insert(0, img)
            writer.writerow(result)

    folder = os.path.join(test_prefix, 'test_stg2')
    imgs = os.listdir(folder)
    imgs.sort()
    with open(csv_name, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for img in imgs:
            result = list(my_inference_model(model, os.path.join(folder, img)))
            result.insert(0, 'test_stg2/' + img)
            writer.writerow(result)

Predict the result of each model (It may take about an hour)

In [ ]:
config_file = 'final/swin.py'
checkpoint_file = 'final/swin.pth'
model = init_model(config_file, checkpoint_file, device='cuda:0')
write_csv(model, 'swin.csv', 'dataset')

load checkpoint from local path: final/swin.pth


In [ ]:
config_file = 'final/resnest101.py'
checkpoint_file = 'final/resnest101.pth'
model = init_model(config_file, checkpoint_file, device='cuda:0')
write_csv(model, 'resnest.csv', 'dataset')

In [ ]:
config_file = 'final/swin.py'
checkpoint_file = 'final/swin_crop.pth'
model = init_model(config_file, checkpoint_file, device='cuda:0')
write_csv(model, 'swin_crop.csv', 'crop')

load checkpoint from local path: final/swin_crop.pth


Do model ensembling, save result into final.csv

In [ ]:
def ensemble(a, b, c):
    a = np.array(a).astype(np.float)
    b = np.array(b).astype(np.float)
    c = np.array(c).astype(np.float)
    output = (a + b + c) / 3
    return list(output)


def read_csv(csv_name):
    with open(csv_name, 'r') as f:
        pred = list(csv.reader(f))
    return pred

output1 = read_csv('swin.csv')
output2 = read_csv('resnest.csv')
output3 = read_csv('swin_crop.csv')


with open('final.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['image', 'ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])
    for i, (a, b, c) in enumerate(zip(output1, output2, output3)):
        if i == 0:
            continue
        result = ensemble(a[1:], b[1:], c[1:])
        result.insert(0, a[0])
        writer.writerow(result)